# Как подготовить визуализацию алгоритма с помощью этого пакета?
Для подготовки визуализации алгоритма нужно сделать три вещи:
* Реализовать процедуру визуализации состояния алгоритма
* Написать сам алгоритм, попутно сохраняя состояния алгоритма
* Воспользоваться `animate_list` из `interactive_visualization.animation_utils`

## Пример #1: быстрая сортировка
Для описания состояния быстрой сортировки достаточно текстовых структур. Здесь можно использовать `IPython.display.Code`, так как он позволяет нормально отображать пробелы и переводы строки. Для описания самого состояния воспользуемся следующими параметрами:
* `array` - массив, который сортируется
* `left, right` - границы текущего рекурсивного вызова
* `x` - разделительный элемент
* `p, q` - текущие значения указателей, которые используются для разделения масссивы на 2 части
В последующем коде `left, right` будут отграничивать подмассив с помощью скобок, а на элементы `p, q` будут указывать стрелочки. 

In [1]:
from IPython.display import Code
def qsort_state(array, left, right, x, p, q):
    extended_array = list(map(str, array[:left])) + ['['] + list(map(str, array[left: right])) + [']'] + list(map(str, array[right:]))
    offset_x = sum(list(map(len, extended_array[:left]))) + left + 2
    zero_line = ''.join([' ' for i in range(offset_x)]) + f'x = {x}'
    first_line = ' '.join(extended_array)
    offset_p = sum(list(map(len, extended_array[:p + 1]))) + p + 1 + len(extended_array[p + 1]) // 2
    offset_q = sum(list(map(len, extended_array[:q + 1]))) + q + 1 + len(extended_array[q + 1]) // 2
    second_line = ''.join([' ' if i != offset_p and i != offset_q else '↑' for i in range(len(first_line))])

    return Code(zero_line + '\n' + first_line + '\n' + second_line)

In [2]:
qsort_state([1, 2, 3, 4, 5, 6], 2, 5, 4, 2, 4)

x = 4
1 2 [ 3 4 5 ] 6
      ↑   ↑

Теперь просто реализуем сам алгоритм попутно добавляя состояния алгоритма описанной процедурой в список, который передаем дополнительным параметром.

In [3]:
import random

def qsort(array, left, right, states):
    if right - left <= 1:
        return
    x = array[random.randint(left, right - 1)]
    p = left
    q = right - 1
    states.append(qsort_state(array, left, right, x, p, q))
    while p <= q:
        while array[p] < x:
            p += 1
            states.append(qsort_state(array, left, right, x, p, q))
        while array[q] > x:
            q -= 1
            states.append(qsort_state(array, left, right, x, p, q))
        if p <= q:
            array[p], array[q] = (array[q], array[p])
            states.append(qsort_state(array, left, right, x, p, q))
            p += 1
            q -= 1
            if p <= q:
                states.append(qsort_state(array, left, right, x, p, q))
    qsort(array, left, q + 1, states)
    qsort(array, p, right, states)   

Теперь запустим нашу функию на каком-нибудь массиве. В качестве `states` нужно передать пустой список.

In [4]:
from interactive_visualization.animation_utils import animate_list

In [5]:
a = [234, 1, 42, 3, 15, 3, 10, 9, 2]
states = []
qsort(a, 0, len(a), states)
animate_list(states, play=True, interval=400);

interactive(children=(IntSlider(value=0, description='step', max=39), Output()), _dom_classes=('widget-interac…

Стоит отметить, что бегунок `steps` является обязательным элементом управления. Проигрыватель и кнопки `назад` и `вперед` являются опциональными и указываются параметрами `play, navigation` (по умолчанию `False, True`).

## Пример #2: Обход в глубину
Более содержательный пример - визуализация обхода в глубину. Здесь мы воспользуемся `graphviz` для отображения графов. Кроме того, здесь будет удобно хранить состояние графа в виде отдельной структуры и немного видоизменять её, вместо того, чтобы каждый раз её строить. Здесь я воспользуюсь оберткой над `graphviz` из `graph_utils`, но можно использовать его и напрямую.

In [6]:
from interactive_visualization.graph_utils import Graph, Arc, Node

В целом описание состояния будет иметь следующий вид:
* Зеленые переходы - те, по которым мы перешли, но еще не вернулись обратно
* Зеленые вершины - те, в которые мы зашли, но еще не вышли из них
* Красные переходы - те, по которым мы перешли и уже вышли
* Красные вершины - те, в которые мы зашли и вышли
* Синие переходы - те, по которым мы не стали переходить, так как они ведут уже в посещенные вершины
В итоге красные переходы должны будут образовать дерево обхода.

In [7]:
def enter_node(node):
    node.SetColor('blue')
    
def enter_arc(node, arc):
    node.SetColor('green')
    arc.attributes['style'] = 'dashed'
    arc.attributes['color'] = 'green'
    
def return_from_arc(node, arc):
    arc.attributes['style'] = 'solid'
    arc.attributes['color'] = 'red'
    node.SetColor('blue')
    
def ignore_arc(arc):
    arc.attributes['color'] = 'blue'
    
def leave_node(node):
    node.SetColor('red')

А теперь сам обход

In [8]:
def dfs(graph, node_id, visited, outlist, path):
    visited.add(node_id)
    path.append(node_id)
    enter_node(graph.nodes[node_id])
    outlist.append(graph.Visualize())
    for arc in graph.nodes[node_id].arcs:
        if arc.end not in visited:
            enter_arc(graph.nodes[node_id], arc)
            dfs(graph, arc.end, visited, outlist, path) 
            return_from_arc(graph.nodes[node_id], arc)
            path.append(node_id)
        else:
            ignore_arc(arc)
        outlist.append(graph.Visualize())      
    leave_node(graph.nodes[node_id])

In [9]:
arcs = [
    Arc(1, 3, 3),
    Arc(1, 4, 7),
    Arc(4, 3, 2),
    Arc(4, 5, 3),
    Arc(1, 5, 2),
    Arc(6, 4, 2),
    Arc(5, 6, 2),
    Arc(6, 7, 1),
    Arc(7, 2, 7),
    Arc(4, 2, 2),
    Arc(3, 2, 5)
]

In [10]:
# Если следующий код выдает ошибку, что ему не удается выполнить `dot`, то
# скорее всего придется отдельно поставить graphviz
# https://graphviz.org/download/
graph = Graph(arcs)
visited = set()
dfs_outlist = []
path = []
dfs_outlist.append(graph.Visualize())
dfs(graph, 1, visited, dfs_outlist, path)
dfs_outlist.append(graph.Visualize())
animate_list(dfs_outlist, play=True, interval=400);

interactive(children=(IntSlider(value=0, description='step', max=19), Output()), _dom_classes=('widget-interac…